In [ ]:
import numpy as np
import open3d as o3d

# Scalable TSDF integration

## TSDF voxels
TSDF (truncated signed distance function) is a useful map representation to fuse and denoise a sequence of Depth/RGB-D images with known camera poses. It voxelizes the 3D space; for each voxel, it computes weight average of signed distance to its closest surface observations from multiple scans. Note this signed distance is **truncated** to preserve details, so that voxels too far away from the surface won't be affected.

By this description, each voxel stores a `float` tsdf value and a `float` or `uint16` weight. At current, we also optionally support `rgb` colors of `float` or `uint16` (we do not use `uint8` to preserve precision during integration). Supported combinations are:
- `float` tsdf, `float` weight
- `float` tsdf, `float` weight, `float` rgb
- `float` tsdf, `uint16` weight, `uint16` rgb

Users may also customize their own voxel types and include other properties (e.g., semantic mask), after modifying the voxels and dispatch pattern in `core/kernel/GeneralEWSharedImpl.h`.

## Voxel blocks and spatial hashing
Initially, in KinectFusion, TSDF is defined in a restricted $512^3$ cube. While this is useful to represent small scenes, it has to sacrifice reconstruction resolution for larger scale environments. VoxelHashing introduces an improved representation where the 3D space is roughly divided into $16^3$ **voxel blocks**. These voxel blocks will only be allocated around surfaces, i.e., around depth observations, and can be indexed dynamically by 3D coordinate using a **hashmap**.

Based on these, we can generate a high resolution volume in such a way:

In [ ]:
voxel_size = 0.008  # voxel resolution in meter
sdf_trunc = 0.04  # truncation distance in meter
block_resolution = 16  # 16^3 voxel blocks
initial_block_count = 1000  # initially allocated number of voxel blocks
device = o3d.core.Device('cuda:0')  # or 'cuda:0' if you have cuda support

volume = o3d.t.geometry.TSDFVoxelGrid(
    {
        'tsdf': o3d.core.Dtype.Float32,
        'weight': o3d.core.Dtype.UInt16,
        'color': o3d.core.Dtype.UInt16
    },
    voxel_size=voxel_size,
    sdf_trunc=sdf_trunc,
    block_resolution=block_resolution,
    block_count=initial_block_count,
    device=device)

Note this voxel size is very high. For smaller scenes (e.g. `stanford/lounge`, `stanford/copyroom`), it should work fine. But for larger scenes (e.g. `stanford/burghers`, `indoor_lidar_rgbd/apartment`), there can be a memory issue. Here are several workarounds if your machine has a limited memory (especially CUDA memory):
- Increase `voxel_size` to 0.01, or a larger value. This will not significantly reduce reconstruction accuracy, but only consumes half the memory.
- Pre-estimate `intial_block_count`. While our internal **hashmap** supports automatic resize when the observed scene is growing, its peak memory consumption is large. If you are able to roughly estimate the number of voxels blocks, rehash will not take place, and there will not be a peak memory issue. As a reference `stanford/lounge` required around 30K voxel blocks, and `indoor_lidar_rgbd/apartment` requires around 80K voxel blocks.


## Input
We then prepare the input, including intrinsics, camera trajectories, and rgbd images for integration. The trajectory file is with the `.log` format that can be generated via the reconstruction system.

In [ ]:
intrinsic = o3d.camera.PinholeCameraIntrinsic(
    o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)

intrinsic = o3d.core.Tensor(intrinsic.intrinsic_matrix, o3d.core.Dtype.Float32,
                            device)


class CameraPose:

    def __init__(self, meta, mat):
        self.metadata = meta
        self.pose = mat

    def __str__(self):
        return 'Metadata : ' + ' '.join(map(str, self.metadata)) + '\n' + \
            "Pose : " + "\n" + np.array_str(self.pose)


def read_trajectory(filename):
    traj = []
    with open(filename, 'r') as f:
        metastr = f.readline()
        while metastr:
            metadata = list(map(int, metastr.split()))
            mat = np.zeros(shape=(4, 4))
            for i in range(4):
                matstr = f.readline()
                mat[i, :] = np.fromstring(matstr, dtype=float, sep=' \t')
            traj.append(CameraPose(metadata, mat))
            metastr = f.readline()
    return traj


camera_poses = read_trajectory("../../test_data/RGBD/odometry.log")

## Integration
Finally we start the integration. Note in this newest version, we are shifting to tensor-based representations, so a conversion is required. Here, `depth_scale` controls the conversion from raw depth images to meter metric depths. Depth max constraints the furthest (and usually the most noisy) points we want to integrate.

In [ ]:
for i in range(len(camera_poses)):
    print("Integrate {:d}-th image into the volume.".format(i))
    color = o3d.io.read_image("../../test_data/RGBD/color/{:05d}.jpg".format(i))
    color = o3d.t.geometry.Image.from_legacy(color, device=device)

    depth = o3d.io.read_image("../../test_data/RGBD/depth/{:05d}.png".format(i))
    depth = o3d.t.geometry.Image.from_legacy(depth, device=device)

    extrinsic = o3d.core.Tensor(np.linalg.inv(camera_poses[i].pose),
                                o3d.core.Dtype.Float32, device)
    volume.integrate(depth, color, intrinsic, extrinsic, 1000.0, 3.0)

## Surface extraction
After TSDF is reconstructed, we can find zero-crossings in the TSDF grid and extract the surfaces. We both support mesh and surface extraction. 
Note surface extraction directly from CUDA is memory consuming. If you want real-time visualization or surface extraction from relatively small scenes, you can directly run from cuda. Otherwise, you may choose to extract mesh after moving it to `.cpu()`. The below implementations showcase both.

In [ ]:
mesh = volume.cpu().extract_surface_mesh().to_legacy()
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh],
                                  front=[0.5297, -0.1873, -0.8272],
                                  lookat=[2.0712, 2.0312, 1.7251],
                                  up=[-0.0558, -0.9809, 0.1864],
                                  zoom=0.47)

In [ ]:
pcd = volume.extract_surface_points().to_legacy()
o3d.visualization.draw_geometries([pcd],
                                  front=[0.5297, -0.1873, -0.8272],
                                  lookat=[2.0712, 2.0312, 1.7251],
                                  up=[-0.0558, -0.9809, 0.1864],
                                  zoom=0.47)